In [1]:
import torch
import requests
import json
import logging

print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: True
Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 5070 Ti


In [2]:
# def prompt_llm(input):
#     prompt = f"""
#     You are a data extraction agent about motorcycle auction.
    
#     Extract the following fields from this motorcycle auction OCR result:
#     Rules:
#     1. address should be in the format of "Kota, Provinsi" or "Kota, Provinsi, Kode Pos".
#     2. price should be below HARGA DASAR and should be a number.
#     3. merk_model should be the brand and model of the motorcycle usually found under 'tahun' (takes 2 lines after 'TAHUN').
#     4. tahun should be the year of the motorcycle, usually after merk_model.
#     5. For CC and odometer:
#     - Look for the word 'WARNA'. The first numeric value after 'WARNA' is the CC.
#     - If no number is found after 'WARNA', set CC to null.
#     - Any numeric value that appears after CC but **before** the section containing 'MESIN' or 'GRADE MESIN' is the odometer.
#     - If no such number is found, set odometer to null.
#     6. grade_mesin should be the engine condition grade (A to G) found near the engine condition section.
#     7. desc_mesin should include all engine-related condition notes exactly as written, up until the 'EKSTERIOR' section, split by ','.
#     8. grade_eksterior should be the exterior condition grade (A to G) found near the exterior section.
#     9. desc_eksterior should include all exterior-related condition notes exactly as written, split by ','.
#     10. Maintain proper JSON formatting.
#     11. Don't include any other text, explanation, or notes in the output.
    
#     Generate the output like this:
#     {{
#     "address": "",
#     "price": "",
#     "merk_model": "",
#     "tahun": "",
#     "CC": "",
#     "odometer": "",
#     "grade_mesin": "",
#     "desc_mesin": "",
#     "grade_eksterior": "",
#     "desc_eksterior": ""
#     }}
    
#     OCR TEXT:
#     \"\"\"{input}\"\"\"
#     """

#     response = requests.post(
#         "http://ollama:11434/api/generate",
#         json={
#             "model": "llama3.2:3b",  # or "llama3:vision" if that's what you're running
#             "prompt": prompt,
#             "stream": False
#         }
#     )

#     try:
#         raw_output = response.json()['response']
#         print("-----------------------------------")  # for debugging
#         print("RAW OUTPUT FROM LLM:\n", raw_output)  # for debugging
#         print("-----------------------------------")  # for debugging
#         parsed_output = json.loads(raw_output)  # <-- parse string to dict
#     except json.JSONDecodeError:
#         logging.error("Failed to parse LLM output as JSON")
#         return None
    
#     return parsed_output

In [8]:
def prompt_llm(input):
  prompt = f"""
  You are a data extraction agent about motorcycle auction.

  Extract the following fields from this motorcycle auction OCR result:
  Rules:
  1. address should be in the format of "Kota, Provinsi" or "Kota, Provinsi, Kode Pos".
  2. price should be below HARGA DASAR and should be a number.
  3. merk_model should be the brand and model of the motorcycle usually found under 'tahun' (takes 2 lines after 'TAHUN').
  4. tahun should be the year of the motorcycle, usually after merk_model.
  5. For CC and odometer:
    - Look for the word 'WARNA'. The first numeric value after 'WARNA' is the CC.
    - If no number is found after 'WARNA', set CC to null.
    - Any numeric value that appears after CC but **before** the section containing 'MESIN' or 'GRADE MESIN' is the odometer.
    - If no such number is found, set odometer to null.
  6. grade_mesin should be the engine condition grade (A to G) found near the engine condition section.
  7. desc_mesin should include all engine-related condition notes exactly as written, up until the 'EKSTERIOR' section, split by ','.
  8. grade_eksterior should be the exterior condition grade (A to G) found near the exterior section.
  9. desc_eksterior should include all exterior-related condition notes exactly as written, split by ','.
  10. Maintain proper JSON formatting.
  11. Don't include any other text, explanation, or notes in the output.
  12. fix the typos in the OCR text.

  Generate the output like this:
  {{
    "address": "",
    "price": "",
    "merk_model": "",
    "tahun": "",
    "CC": "",
    "odometer": "",
    "grade_mesin": "",
    "desc_mesin": "",
    "grade_eksterior": "",
    "desc_eksterior": ""
  }}

  OCR TEXT:
  \"\"\"{input}\"\"\"
  """

  response = requests.post(
        "http://ollama:11434/api/generate",
        json={
            "model": "qwen2.5:7b-instruct-q3_K_S",  # or "llama3:vision" if that's what you're running
            "prompt": prompt,
            "stream": False
        }
    )
  return response


In [9]:
structured_data = prompt_llm( ['LOT', 'Jakarta Raya - Eksklusif MAF MCF NONJAWA Motor', '1', 'JAKARTA', 'RAYA MOTOR', 'JLHusein Sastra Negara', 'Komplek Pergudangan Nusa Indah', 'Jurumudi.', 'HARGA DASAR', '15,000,000', '+1.1%', 'NOTE TAMBAHAN', 'MERK', 'MODEL', 'NO. POLISI', 'TAHUN', 'HONDA', 'SCOOPY 110', 'BP 6851 HQ', '2024', 'NOTA PAJAK (STNK)', 'NOMOR MESIN', 'NOMOR RANGKA', 'TDK', 'JMO4E2183737', 'MHIJMO4ZIRK184011', 'Cc', 'KUNCI', 'REMOTE', 'ACCU', 'ODOMETER', 'WARNA', '1 /ADA', 'ADA', '5.178', 'HITAM', 'MESIN', '1', 'Kondisi Kick Starter Matic CVT Berisik', 'B', 'Kondisi Box Filter Saringan Udara Gores', 'Kondisi Bak Cvt Gores Kecil', 'EKSTERIOR', '509', '509', 'Velg Depan Gores Kecil', 'B', 'Deck Step Gores Sedang', '7005000', 'Tameng Bawah Gores Sedang', 'Cover Rangka Tengah Gores Kecil', 'KETERANGAN', 'GORES (9): PECAH (P): KARAT ( <) RETAK(R; imitasi ( Iai', 'KF(:II', '{1 "FienG 1!', 'K-Sak (3)', 'IlJAK A12a ! |,a)', 'CATATAN', 'JBA hanya melakukan inspeksi secara statis', '2 Wajib melakukan pengecekan unit dan dokumen sebelum mengikuti lelang', 'Halaman 1', 'Tanggal cetak: 09/05/2025 15.01 wib', 'Rp.'])
print(structured_data.json()["response"])

{
    "address": "JAKARTA RAYA MOTOR, JAKARTA",
    "price": "15,000,000",
    "merk_model": "HONDA SCOOPY 110",
    "tahun": "2024",
    "CC": null,
    "odometer": "5.178",
    "grade_mesin": "B",
    "desc_mesin": "Kondisi Kick Starter Matic CVT Berisik,Kondisi Box Filter Saringan Udara Gores,Kondisi Bak Cvt Gores Kecil",
    "grade_eksterior": "509",
    "desc_eksterior": "Velg Depan Gores Kecil,B Deck Step Gores Sedang,Tameng Bawah Gores Sedang,Cover Rangka Tengah Gores Kecil"
}


In [9]:
response = requests.post(
    "http://ollama:11434/api/generate",
    json={
        "model": "llama3.2:3b",
        "prompt": "hi",
        "stream": False
    }
)
print(response.json()["response"])

Hello! How can I assist you today?


In [3]:
print(torch.__version__)

2.8.0.dev20250505+cu128


In [4]:
x = torch.randn(3, 3).cuda()
print("Tensor on GPU:", x)


Tensor on GPU: tensor([[ 0.6151,  1.2455, -1.4923],
        [ 0.6613, -0.6075,  0.7909],
        [-0.9400,  0.2100,  0.5381]], device='cuda:0')


In [8]:
import requests
from bs4 import BeautifulSoup



# Send HTTP request to the URL
headers = {
    "User-Agent": "Mozilla/5.0"  # Helps avoid blocks
}
response = requests.get(url, headers=headers)
print(response)

# Check if the request succeeded
if response.status_code == 200:
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')



<Response [200]>


In [37]:
import requests
from bs4 import BeautifulSoup
import re

# Base URL without the page number
base_url = 'https://www.jba.co.id'
headers = {'User-Agent': 'Mozilla/5.0'}

for page_num in range(1, 2):
    url = base_url + '/id/lelang-motor/search?keyword=&vehicle_type=bike&page='+ str(page_num)
    print(f"\nScraping page {page_num} - {url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch page {page_num}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')

    product_list = soup.find('div', class_='product-list')
    if not product_list:
        print("No product list found on this page.")
        continue

    rows = product_list.find('div', class_='row')
    if not rows:
        print("No rows found in product list.")
        continue

    for item in rows.find_all('div', class_='product-item'):
        a_tag = item.find('a')
        if a_tag and a_tag.get('href'):
            # url_motor = base_url + a_tag.get('href')
            url_motor = base_url+'/id/lelang-motor/search-detail/4117/640401'
            response = requests.get(url_motor, headers=headers)
            if response.status_code != 200:
                print(f"Failed to fetch {url_motor}")
                continue
            soup = BeautifulSoup(response.text, 'html.parser')
            for img_tag in soup.find_all('img'):
                img_src = img_tag.get('src')
                if img_src and pattern.search(img_src):
                    print("Matched image:", img_src)
            break
            # print(url_motor)



Scraping page 1 - https://www.jba.co.id/id/lelang-motor/search?keyword=&vehicle_type=bike&page=1
Matched image: https://img.jba.co.id/upload/jbabike/640401/PNK0005919-250424112852-1.jpg


In [40]:
test_path = '/home/jovyan/JMB0007280-250429170107-1.jpg'

import easyocr
reader = easyocr.Reader(['id'])

result = reader.readtext(test_path)

for (bbox, text, prob) in result:
    print(text)

LOT
Jambi
Motor 100
08, May 2025
301
JAMBI - JI.Lingkar Selatan RT 27 Kelurahan Paal
Merah, Kec.Jambi Selatan, Kota Jambi 36128
HARGA DASAR
8,000,000
+1.1%
NOTE TAMBAHAN
WAJIB CEK UNIT
1&
DOKUMEN  SECARA LANGSUNG T1 WAJIB
CEK KEABSAHAN
UNIT I(CEK SAMSATICEK
BLOKIRI) //
MERK
MODEL
NO. POLISI
TAHUN
HONDA
SCOOPY CBS ISS 110
BH 3442 OZ
2024
NOTA PAJAK (STNK)
NOMOR MESIN
NOMOR RANGKA
TDK
JMO4E2011670
MHIJMO424RKOO9056
Cc
KUNCI
REMOTE
ACCU
ODOMETER
WARNA
110
TIA /
MATI
HITAM
MESIN

Kondisi Mesin Mati
F
Kondisi Odometer Tidak Terlihat Imati
Keberadaan Kunci Tidak Ada
Kondisi Steering Lock Rusak
Kondisi Box Filter Saringan Udara Gores
Kondisi Bak Cvt Gores Sedang

259
259
EKSTERIOR
Jok Tombol Otomatis Rusak
D
Tameng Bawah Pecah Sedang
Cover Rangka Tengah Gores Sedang
Jok Sobek Besar
Plat Nomor
Belakang Tidak Ada
Shockbreaker Belakang Karat Kecil
Deck
Gores Sedang
Depan Gores Kecil
Spakbor Depan Penyok Besar
Tameng Bawah Gores Sedang
Kondisi Rem
Tangan Kanan Bengkok
Kondisi
Kaca Spion Tidak Ada